Data from:
https://www.kaggle.com/seattle-public-library/seattle-library-checkout-records/data

We will clean the data so as to  predict # checkouts per each item per 2 week.
* filter+group/count data by Bibnum (this may lose different versions of same item, but is cleaner than isbn at least). i.e drop items that are super rare

In [ ]:
import pandas as pd

In [ ]:
# don't parse dates at first; and note that initial filtering is per withdrawal, not unique day/date (will refilter later)

# df = pd.concat([pd.read_csv("Checkouts_By_Title_Data_Lens_2017.csv.gz",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 30)
#              ,pd.read_csv("Checkouts_By_Title_Data_Lens_2016.csv.gz",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 40)
#              ,pd.read_csv("Checkouts_By_Title_Data_Lens_2015.csv.gz",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 40)
#              ,pd.read_csv("Checkouts_By_Title_Data_Lens_2014.csv.gz",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 40)

#                ])

df = pd.concat([pd.read_csv("../input/Checkouts_By_Title_Data_Lens_2017.csv",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 30)
             ,pd.read_csv("../input/Checkouts_By_Title_Data_Lens_2016.csv",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 40)
             ,pd.read_csv("../input/Checkouts_By_Title_Data_Lens_2015.csv",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 40)
             ,pd.read_csv("../input/Checkouts_By_Title_Data_Lens_2014.csv",usecols=["BibNumber","CheckoutDateTime"]).groupby("BibNumber").filter(lambda x: len(x) > 40)
               ])

print(df.shape)
df.tail()

In [ ]:
# get count of uniques per col
df.apply(lambda x: len(x.unique()))

The apply is X3 faster! (8 vs 30 ms per loop)
* %%timeit
    * slow: df2.CheckoutDateTime.str.split(" ",1,expand=True)[0]
    * faster: df2.CheckoutDateTime.apply(lambda x: x.split(" ")[0])

In [ ]:
%%timeit
# df2["date"] = df2.CheckoutDateTime.str.split(" ",1,expand=True)[0]


In [ ]:
#%%timeit
# df2["date"] = df2.CheckoutDateTime.apply(lambda x: x.split(" ")[0])
df2.CheckoutDateTime.apply(lambda x: x.split(" ")[0])

In [ ]:
# %%timeit
# df2.CheckoutDateTime.apply(lambda x: pd.to_datetime(x.split(" ")[0],format="%m/%d/%Y"))

In [ ]:
df.CheckoutDateTime = df.CheckoutDateTime.apply(lambda x: x.split(" ")[0])

In [ ]:
# get count of uniques per col
df.apply(lambda x: len(x.unique()))

In [ ]:
df.tail()

In [ ]:
# df2 = df.head(100000).sample(7000)
# df2.CheckoutDateTime = pd.to_datetime(df2.CheckoutDateTime,infer_datetime_format=True)
# df2.CheckoutDateTime.dt.date.head()

# # 
# df2.groupby(['BibNumber',pd.Grouper(key='CheckoutDateTime', freq='W')])['BibNumber'].agg('count')

# ## round timestamps to date:
### df2.CheckoutDateTime = df2.CheckoutDateTime.dt.date

### df2['CheckoutDateTime'].dt.floor('d')

# possibly - rounds to nearest week:
### df2['CheckoutDateTime'] - pd.to_timedelta(df2['CheckoutDateTime'].dt.dayofweek, unit='d')

http://pbpython.com/pandas-grouper-agg.html
* Pandas groupby, grouper, on key + date:

    * df.groupby(['BibNumber',pd.Grouper(key='CheckoutDateTime', freq='W')])['BibNumber'].agg('count') 
    
        * W = week. SMS, MS  (SMS: 	semi-month start frequency (1st and 15th))
        * http://pandas.pydata.org/pandas-docs/stable/api.html#datetimelike-properties
        * datetime offset aliases (e.g. middle + end of month: MS):
        https://pandas.pydata.org/pandas-docs/stable/timeseries.html


* Warning: this may take a while or get stuck, partially due to parsing as datetime first! 
    
* After this we can filter for items which weren't checked out enough times

* DAtetime Round: 
    https://stackoverflow.com/questions/32344533/how-do-i-round-datetime-column-to-nearest-quarter-hour
    * df2.CheckoutDateTime = df2.CheckoutDateTime.dt.date
    * df2['CheckoutDateTime'].dt.floor('d')
    
    
    
* datetime parse made much faster if we explicitly give format + drop unnneeded hour/minute components: 
    * https://stackoverflow.com/questions/38721087/split-timestamp-column-into-two-seperate-date-and-time-columns-with-python
    
    * more speedup tricks: 
    https://stackoverflow.com/questions/19704338/why-this-date-conversion-with-pandas-to-datetime-is-much-slower-than-some-altern
   

In [ ]:
print(df.shape)

In [ ]:
df.CheckoutDateTime = pd.to_datetime(df.CheckoutDateTime,format="%m/%d/%Y", infer_datetime_format=True)

In [ ]:
df.head()

### groupby item, resample, get count:

* filter out some rarer items (note that the frequency matters - we're looking at unique occurences per item per date range now..) 

In [ ]:
df = df.groupby(['BibNumber',pd.Grouper(key='CheckoutDateTime', freq='SMS')])['BibNumber'].agg('count')

In [ ]:
df.shape

In [ ]:
df.groupby("BibNumber").filter(lambda x: len(x) > 30).shape

In [ ]:
df.groupby("BibNumber").filter(lambda x: len(x) > 24).shape

In [ ]:
df = df.groupby("BibNumber").filter(lambda x: len(x) > 12)
print(df.shape)
df.head()

### note that grouped df doesn't have col headers 


* data has about 67K unique BibNumbers , 2.9 million rows, 

In [ ]:
# df.to_csv("SeattleLibrary_Sum_BiMonthly_Checkouts_2017_2014.csv.gz", compression="gzip")
df.to_csv("SeattleLibrary_Sum_BiMonthly_Checkouts_2017_2014.csv")
### added columns manually to the csv with a text editor

In [ ]:
df = pd.read_csv("SeattleLibrary_Sum_BiMonthly_Checkouts_2017_2014.csv",names=['BibNumber', 'CheckoutDateTime', 'counts'])
df.head()

In [ ]:
df.to_csv("SeattleLibrary_Sum_BiMonthly_Checkouts_2017_2014.csv.gz",index=False,compression="gzip")

In [ ]:
df.shape

--------------------------------
### item metadata (lookup)

In [ ]:
df_meta = pd.read_csv("../input/Library_Collection_Inventory.csv").drop(["ISBN","ReportDate"],axis=1)  # report date is just 2 dates. may want ot keep ISBNs? 

print(df_meta.shape)

# Some cleanup: note that there can be multiple publication years in the col (or no year)! This is just a quick hack:
# some historical dates will fail due to pandas limitation on parsing (e.g. 1416). 
# we extract the year into another column and hack a date onto it ("01-01"+year )
df_meta["PublicationYearDate"] =  "01/01/"+ df_meta.PublicationYear.str.extract('(\d{4})')
## orig: #  df_meta.PublicationYear = pd.to_datetime(df_meta.PublicationYear.str.extract('(\d{4})'),format="%Y") # full, failed for some cases

# df_meta.PublicationYear = df_meta.PublicationYear.str.extract('(\d{4})').astype(int) # fails due to nans (can do .astype(float) instead)

# replace single stre with boolean
df_meta.FloatingItem.replace("Floating",True,inplace=True)

In [ ]:
df_meta.head()

In [ ]:
# df_meta.loc[df_meta.PublicationYear<1500].head()

In [ ]:
# get count of uniques per col
df_meta.apply(lambda x: len(x.unique()))

In [ ]:
df_meta.info()

In [ ]:
Bibs = set(df.BibNumber)
len(Bibs)

In [ ]:
len(df_meta.BibNum.unique())
# only ~12% of our catalogue is relevant to the "popular" items

In [ ]:
df_meta.to_csv("Library_Collection_Inventory_lookup.csv.gz",compression="gzip",index=False)

------------------------------------------------

unnneeded: 

* could add week-year manuually for indexing, regoruping.

* additional alt: https://stackoverflow.com/questions/38792122/how-to-group-and-count-rows-by-month-and-year-using-pandas

In [ ]:
# df['Week-Year'] = df['CheckoutDateTime'].apply(lambda x: "%d/%d" % (x.week, x.year))
# df.groupby(['Week-Year', 'BibNumber']).size()

In [ ]:
# df.groupby(['BibNumber', pd.Grouper(key='CheckoutDateTime', freq='W')]).sum()
# df.groupby('Week-Year').BibNumber.value_counts()

# df.groupby('Week-Year').BibNumber.value_counts().to_csv("wy.csv", date_format='%Y%m%d')

---------------------------